In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
print("ok")

ok


In [3]:
!pip install langchain_google_genai
!pip install langchain pypdf pinecone-client google-genai google 
!pip install -U langchain-community
! pip install tiktoken
! pip install google pinecone
! pip install langchain_pinecone
! pip install -U langchain-huggingface

In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter # use for general models
# from langchain_google_genai import ChatGoogleGenerativeAI
import os
import tiktoken
from dotenv import load_dotenv
import pinecone

c:\Users\HP\anaconda3\envs\medic_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

In [6]:
from tqdm.autonotebook import tqdm

In [7]:
import time
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

load_dotenv()



pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [8]:
pc = Pinecone(api_key = pinecone_api_key)

index_name = "medical-chatbot-with-llama2" 

In [ ]:
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        deletion_protection="enabled",  # Defaults to "disabled"
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

In [11]:
def load_pdf(file_path):
    loader = DirectoryLoader(file_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [13]:
extracted_data = load_pdf("data/")

In [16]:
def chunk_data(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200) # Initialize the text splitter with a chunk size of 1000 characters and an overlap of 200 characters
    text_chunks = text_splitter.split_documents(extracted_data) # Split the documents into smaller chunks

    return text_chunks

In [17]:
text_chunks = chunk_data(extracted_data)
print(f"Number of chunks: {len(text_chunks)}") # Print the number of chunks created

Number of chunks: 3424


In [13]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [20]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
query = embeddings.embed_query("What is the role of the pancreas in digestion?")
print(f"Query vector: {query[:5]}")  # Print the first 5 elements of the query vector
print(f"Query vector length: {len(query)}")  # Print the length of the query vector

Query vector: [0.06813713163137436, 0.023588644340634346, -0.05461941286921501, 0.011025440879166126, -0.03159325569868088]
Query vector length: 384


In [15]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(
    index=index, 
    embedding=embeddings)

In [25]:
vector_store.add_documents(text_chunks)

['0f8cf881-bede-45c5-b67f-04ae09e311d1',
 '8b737ede-94fc-4004-b420-723352b24f75',
 '036a4093-7af3-42a3-8869-0f44bed9489e',
 '4a2211a6-dfc6-4bff-b348-121e531b389a',
 '18c076f4-4519-480c-9d33-409e4435972c',
 'b89f4912-09c1-45eb-8555-d0d65b10a999',
 'bf176341-8e7e-495d-ace8-3d0b602be371',
 'b0292d9e-3e8f-4f1f-bcc8-7a07db8caafa',
 '92a3fad0-680d-4c9b-91b2-504accdaeeb3',
 'f5c13342-8772-47c5-924c-f32492b633f0',
 'c15b6000-715a-4ec8-9893-87671e2d3274',
 'c681c3fb-c5e2-4cf7-9364-4f80de4780c7',
 '601c4923-9b9d-42bb-945b-a212ed5e1195',
 'ba5c12a0-fbff-4530-b51e-83810dac91ea',
 'd68716ec-2ec9-4027-ac7f-84fe36f35798',
 '7eaa71d9-654b-4333-b9d1-623a5f0c184f',
 '9ebf3b99-1d2c-434a-ba67-07de28d09048',
 'dec1c840-c12c-466d-93bb-8cbc6363bc85',
 '720fa369-1079-499b-bf02-2136457cd4ca',
 '51c77c09-e926-46f2-a7d8-91356e3b7b73',
 'be6a17fe-7232-45ce-8c94-4f9c7be8cbc8',
 '39fc1801-096b-4068-8fe6-60f872766a64',
 '1b5efa7c-6d4c-4578-a77c-5d24eaea2d1d',
 '37627574-20ef-49c9-87eb-0bb5aeede881',
 '75c84bf7-420a-

In [26]:
query = "What is the role of the pancreas in the human body?"

results = vector_store.similarity_search_with_score(query=query, k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.480328] in children, occurring once every 10,000–15,000 live
births. Half of all liver transplants are done for this reason.
The normal anatomy of the bile system begins with-
in the liver, where thousands of tiny bile ducts collect
bile from liver cells. These ducts merge into larger and
larger channels, like streams flowing into rivers, until
they all pour into a single duct that empties into the duo-
denum (first part of the small intestine). Between the
liver and the duodenum this duct has a side channel con-
nected to the gall bladder. The gall bladder stores bile
and concentrates it, removing much of its water content.
Then, when a meal hits the stomach, the gall bladder
contracts and empties its contents.
Bile is a mixture of waste chemicals that the liver
removes from the circulation and excretes through the
biliary system into the intestine. On its way out, bile
assists in the digestion of certain nutrients. If bile cannot
get out because the channels are absent or bl

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

# use this code only if you have downloaded the model and stored it in the model folder.
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          "context_length": 2048, 
                          'temperature':0.8,
                          "local_files_only": True})


# use this commented code rather than the one above if you haven't downloaded the model before. this will help download the model locally

# llm = CTransformers(
#     model="TheBloke/Llama-2-7B-Chat-GGML",  # Folder path, not the full .bin file
#     model_file="llama-2-7b-chat.ggmlv3.q4_0.bin",  # Actual model filename
#     model_type="llama",
#     config={
#         "max_new_tokens": 512,
#         "temperature": 0.8,
#     }
# )


retriever = vector_store.as_retriever()

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [20]:
query = "What is the role of the pancreas in the human body?"

In [ ]:
response = qa({"query": query})

# Step 4: Print the result
print(response["result"])

In [24]:
response = qa.invoke({"query": query})

# Step 4: Print the result
print(response["result"])

The pancreas plays a crucial role in the human body's digestive system by producing digestive enzymes that help break down food into nutrients for absorption, as well as hormones such as insulin and glucagon that regulate blood sugar levels.
